In [2]:
import sys

sys.path.append("..")
%load_ext autoreload
%autoreload 2
from trl_wrapper.trainer_wrapper import TrainerWrapper, WRITING_GRPO_CONFIG
from trl_wrapper.wrapper_config import QWEN_3_600M

cfg = WRITING_GRPO_CONFIG
cfg.train_batch_size = 2
cfg.num_generations = 2
cfg.model_id_or_path = QWEN_3_600M
cfg.max_completion_length = 32
cfg.max_prompt_length = 32
cfg.max_samples = 10

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
wrapper = TrainerWrapper(cfg, True)

2025-04-28 20:35:35.818 | INFO     | trl_wrapper.trainer_wrapper:__init__:302 - Setting padding side to: left


In [4]:
wrapper.init_data_module()

Map: 100%|██████████| 1184/1184 [00:00<00:00, 27560.57 examples/s]
2025-04-28 20:35:39.337 | INFO     | synthetic_data.creative_writing_bench.slop_score:load_slop_list_to_set:20 - Loaded 1000 items from ../dataset_files//slop_list.json
2025-04-28 20:35:39.337 | INFO     | synthetic_data.creative_writing_bench.slop_score:load_slop_list_to_set:20 - Loaded 200 items from ../dataset_files//slop_list_bigrams.json
2025-04-28 20:35:39.338 | INFO     | synthetic_data.creative_writing_bench.slop_score:load_slop_list_to_set:20 - Loaded 200 items from ../dataset_files//slop_list_trigrams.json


In [7]:
wrapper.init_model()

2025-04-28 20:36:36.320 | INFO     | trl_wrapper.trainer_wrapper:init_model:317 - Using device: mps
2025-04-28 20:36:36.321 | INFO     | trl_wrapper.trainer_wrapper:init_model:331 - Loading model Qwen/Qwen3-0.6B with attn_impl: sdpa


In [ ]:
wrapper.data_module.config.run_mode

In [9]:
wrapper.init_trainer()

2025-04-28 20:36:46.542 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:395 - Saving output to: ./runs/04-28-20-36-767378-qwen3-0.6b-
2025-04-28 20:36:46.543 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:418 - Initializing trainer, run_name: 04-28-20-36-767378-qwen3-0.6b-, wandb project: writing-grpo
2025-04-28 20:36:46.544 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:501 - Using vllm: False


In [ ]:
wrapper.train()

In [14]:
import pandas as pd
dataset_pd = pd.read_parquet(
    "../dataset_files/gutenberg_backtranslate_from_txt.parquet"
)
completion, instruction = dataset_pd.loc[500]['text'], dataset_pd.loc[500]['instruction']

In [ ]:
max_score = 0
for i in range(10000):
    completion = dataset_pd.loc[i]['text']
    score = wrapper.data_module.bench.calculate_slop_index(completion)
    if score > max_score:
        max_score = score
        print(score)
print(max_score)

In [ ]:
wrapper.data_module.reward_functions()[0]([instruction], [completion])